In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 1.6 MB/s eta 0:00:00


In [2]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/analyze_final_model')

####  use the prediction model to predict the results in the test dataset (validate the model performance)

In [3]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560_phylo')

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

import warnings
from tqdm import tqdm
import os
from pathlib import Path
# Define the device
device = "cuda" if torch.cuda.is_available() else "mps" if  torch.backends.mps.is_available() else "cpu" # torch.has_mps or
print("Using device:", device)
if (device == 'cuda'):
    print(f"Device name: {torch.cuda.get_device_name(device.index)}")
    print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
elif (device == 'mps'):
    print(f"Device name: <mps>")
else:
    print("NOTE: If you have a GPU, consider using it for training.")
    print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
    print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
device = torch.device(device)

Using device: cuda
Device name: NVIDIA A100-SXM4-40GB
Device memory: 39.56427001953125 GB


In [5]:
ESP_test_df_enzy = torch.load('ESP_test_df_enzy.pt')
# Load the saved embeddings_results
ESP_test_df_smiles = torch.load('ESP_test_df_smiles.pt')
y_test = torch.load('ESP_test_df_label.pt')
ESP_test_df_enzy_add = torch.load('ESP_test_df_enzy_>2800_<8000.pt')
ESP_test_df_smiles_add = torch.load('ESP_test_df_smiles_>2800_<8000.pt')
y_test_add = torch.load('ESP_test_df_label_>2800_<8000.pt')

ESP_test_df_enzy = torch.cat([ESP_test_df_enzy, ESP_test_df_enzy_add], dim=0)
ESP_test_df_smiles = torch.cat([ESP_test_df_smiles, ESP_test_df_smiles_add], dim=0)
y_test = torch.cat([y_test, y_test_add], dim=0)
print(ESP_test_df_enzy.shape, ESP_test_df_smiles.shape, y_test.shape)

test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)


<ipython-input-5-b8ec678e0306>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy.pt')
<ipython-input-5-b8ec678e0306>:3: Future

torch.Size([13336, 2560]) torch.Size([13336, 768]) torch.Size([13336, 1])


In [6]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [7]:
loss_fn = nn.MSELoss().to(device)

In [38]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc , tn,fp,fn,tp  # , precision, sensitivity, recall, MCC, AUC, f1


In [9]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560_phylo')

In [10]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_esm2_2560_fine_tuned_add_>2800_<8000__ACC=0.9476.pt'
# Load the entire model
model_test = torch.load(load_path)
run_validation(model_test,test_loader,loss_fn, device)

<ipython-input-10-ff1e0fd3c066>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_test = torch.load(load_path)


loss_sum=  0.049867110144167164 ACC=  0.9476604679064187 bacc=  0.9170804862959899 precision=  0.943639798488665 specificity=  0.9817718940936864 sensitivity=  0.8523890784982935 recall=  0.8523890784982935 MCC=  0.8628651785864061 AUC=  0.9652937024576622 f1=  0.8956963538553496


(0.049867110144167164,
 0.9476604679064187,
 0.9170804862959899,
 9641,
 179,
 519,
 2997)

In [11]:
## looks great because the loaded model is the correct model we plan to use for evaluation

In [12]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return acc*100, MCC, AUC   tn,fp,fn,tp # , precision, sensitivity, recall, MCC, AUC, f1


#### ESM-2-2560 and MolFormer embeddings function

In [25]:
!pip install fair-esm
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 49.5 MB/s eta 0:00:00


In [26]:
def esm_embeddings_2560(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != esm2_alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t12_35M_UR50D' only has 12 layers, and therefore repr_layers parameters is equal to 12
      results = esm2(batch_tokens, repr_layers=[36], return_contacts=False)
  token_representations = results["representations"][36].cpu()
  del results, batch_tokens
  torch.cuda.empty_cache()
  gc.collect()
  return token_representations[:,1:-1,:].mean(1)


In [27]:
import torch
from transformers import AutoModel, AutoTokenizer
from rdkit import Chem

model_smiles = AutoModel.from_pretrained("ibm/MoLFormer-XL-both-10pct", deterministic_eval=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ibm/MoLFormer-XL-both-10pct", trust_remote_code=True)

def MolFormer_embedding(model_smiles, tokenizer, SMILES_list):
    inputs = tokenizer(SMILES_list, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model_smiles(**inputs)
    # NOTICE: if you have several smiles in the list, you will find the average embedding of each token will remain the same
    #           no matter which smiles in side the list, however, the padding will based on the longest smiles,
    #           therefore, the last hidden state representation shape:[len, 768] will change for the same smiles in difference smiles list.
    return outputs.pooler_output # shape is [len_list, 768] ; torch tensor;

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_molformer.py:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- configuration_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_molformer.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

tokenization_molformer.py:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer_fast.py
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.json:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

### divide the test dataset

In [3]:
import pandas as pd
import numpy as np
ESP_test_df = pd.read_csv('ESP_test_df.csv', header= 0)
print(ESP_test_df.shape)

(13336, 5)


In [4]:
enzy_sequence_40 = pd.read_excel('all_sequences_clustered_sequences_40.xlsx', header= 0)
enzy_sequence_60 = pd.read_excel('all_sequences_clustered_sequences_60.xlsx', header= 0)
enzy_sequence_80 = pd.read_excel('all_sequences_clustered_sequences_80.xlsx', header= 0)
print(enzy_sequence_40.shape,enzy_sequence_60.shape,enzy_sequence_80.shape)


(5628, 1) (7676, 1) (8914, 1)


In [ ]:
df_enzy_40_pairs = pd.DataFrame()
df_enzy_60_pairs = pd.DataFrame()
df_enzy_80_pairs = pd.DataFrame()
# extract the row with 40 similarity seqeunces
for seq in enzy_sequence_80['sequence']:
    count = ESP_test_df[ESP_test_df['Protein sequence'] == seq].shape[0] # check whether the seqeunce is in the test dataset
    # print(count)
    # print(ESP_test_df[ESP_test_df['Protein sequence'] == seq])
    if count > 0 : # if this sequence is existed
        df_enzy_80_pairs = pd.concat([df_enzy_80_pairs,ESP_test_df[ESP_test_df['Protein sequence'] == seq]],axis = 0)

# extract the row with 40 similarity seqeunces
for seq in enzy_sequence_60['sequence']:
    count = ESP_test_df[ESP_test_df['Protein sequence'] == seq].shape[0] # check whether the seqeunce is in the test dataset
    # print(count)
    # print(ESP_test_df[ESP_test_df['Protein sequence'] == seq])
    if count > 0 : # if this sequence is existed
        df_enzy_60_pairs = pd.concat([df_enzy_60_pairs,ESP_test_df[ESP_test_df['Protein sequence'] == seq]],axis = 0)

# extract the row with 40 similarity seqeunces
for seq in enzy_sequence_40['sequence']:
    count = ESP_test_df[ESP_test_df['Protein sequence'] == seq].shape[0] # check whether the seqeunce is in the test dataset
    # print(count)
    # print(ESP_test_df[ESP_test_df['Protein sequence'] == seq])
    if count > 0 : # if this sequence is existed
        df_enzy_40_pairs = pd.concat([df_enzy_40_pairs,ESP_test_df[ESP_test_df['Protein sequence'] == seq]],axis = 0)


In [ ]:
df_only_less_40 = df_enzy_40_pairs[['Protein sequence', 'SMILES']]
df_only_40_60 = []

for i in range(df_enzy_60_pairs.shape[0]):
    seq = df_enzy_60_pairs['Protein sequence'].iloc[i]
    count = df_enzy_40_pairs[df_enzy_40_pairs['Protein sequence'] == seq].shape[0]
    if count == 0 :
        df_only_40_60.append(df_enzy_60_pairs.iloc[i])
df_only_40_60 = pd.DataFrame(df_only_40_60)


In [ ]:
df_only_less_40 = df_enzy_40_pairs[['Protein sequence', 'SMILES']]
df_only_60_80 = []

for i in range(df_enzy_80_pairs.shape[0]):
    seq = df_enzy_80_pairs['Protein sequence'].iloc[i]
    count = df_enzy_60_pairs[df_enzy_60_pairs['Protein sequence'] == seq].shape[0]
    if count == 0 :
        df_only_60_80.append(df_enzy_80_pairs.iloc[i])

df_only_60_80 = pd.DataFrame(df_only_60_80)

In [ ]:
df_only_less_40 = df_enzy_40_pairs
df_only_less_40.drop_duplicates().to_csv('enzyme_sequence_lower_40.csv')
df_only_40_60.drop_duplicates().to_csv('enzyme_sequence_40_60.csv')
df_only_60_80.drop_duplicates().to_csv('enzyme_sequence_60_80.csv')

### further divide the dataset based on the small molecule occurance

In [14]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/analyze_final_model')

In [15]:
import pandas as pd
df_only_less_40 = pd.read_csv('enzyme_sequence_lower_40.csv')
df_only_40_60 = pd.read_csv('enzyme_sequence_40_60.csv')
df_only_60_80 = pd.read_csv('enzyme_sequence_60_80.csv')
print(df_only_less_40.shape)
print(df_only_40_60.shape)
print(df_only_60_80.shape)


(6396, 6)
(2708, 6)
(1653, 6)


In [16]:
# read the training dataset
ESP_train_df = pd.read_csv('ESP_train_df.csv', header= 0)


### enzyme_sequence_lower_40

#### split the dataset

In [40]:
# extract all the smiles
df_0 = pd.DataFrame()
df_1 = pd.DataFrame()
df_2 = pd.DataFrame()
df_3 = pd.DataFrame()
df_4 = pd.DataFrame()
df_5 = pd.DataFrame()
df_6 = pd.DataFrame()
df_7 = pd.DataFrame()
df_8 = pd.DataFrame()
df_9 = pd.DataFrame()
df_10 = pd.DataFrame()
df_11 = pd.DataFrame()
a = 0
a_0 = 0
a_1 = 0
a_2 = 0
a_3 = 0
a_4 = 0
a_5 = 0
a_6 = 0
a_7 = 0
a_8 = 0
a_9 = 0
a_10 = 0
a_11 = 0
for smiles in df_only_less_40['SMILES']:
    count = ESP_train_df[ESP_train_df['SMILES'] ==smiles].shape[0] # just check whether the smiles exist in the ESP_train dataset
    a = a + 1
    if count == 0: # means the smiles have not occur in the train dataset.
        df_0 = pd.concat([df_0,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_0 = a_0 + 1
    elif 0 < count <11 :
        df_1 = pd.concat([df_1,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_1 = a_1 + 1
    elif count > 10:
        df_11 = pd.concat([df_11,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_11 = a_11 + 1

In [41]:
print(df_0.shape, df_1.shape, df_11.shape)

(331, 6) (442, 6) (5623, 6)


In [42]:
# # extract all the smiles
# df_0 = pd.DataFrame()
# df_1 = pd.DataFrame()
# df_2 = pd.DataFrame()
# df_3 = pd.DataFrame()
# df_4 = pd.DataFrame()
# df_5 = pd.DataFrame()
# df_6 = pd.DataFrame()
# df_7 = pd.DataFrame()
# df_8 = pd.DataFrame()
# df_9 = pd.DataFrame()
# df_10 = pd.DataFrame()
# df_11 = pd.DataFrame()
# a = 0
# a_0 = 0
# a_1 = 0
# a_2 = 0
# a_3 = 0
# a_4 = 0
# a_5 = 0
# a_6 = 0
# a_7 = 0
# a_8 = 0
# a_9 = 0
# a_10 = 0
# a_11 = 0
# for smiles in df_only_less_40['SMILES']:
#     count = ESP_train_df[ESP_train_df['SMILES'] ==smiles].shape[0] # just check whether the smiles exist in the ESP_train dataset
#     a = a + 1
#     if count == 0: # means the smiles have not occur in the train dataset.
#         df_0 = pd.concat([df_0,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_0 = a_0 + 1
#     elif count == 1:
#         df_1 = pd.concat([df_1,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_1 = a_1 + 1
#     elif count == 2:
#         df_2 = pd.concat([df_2,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_2 = a_2 + 1
#     elif count == 3:
#         df_3 = pd.concat([df_3,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_3 = a_3 + 1
#     elif count == 4:
#         df_4 = pd.concat([df_4,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_4 = a_4 + 1
#     elif count == 5:
#         df_5 = pd.concat([df_5,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_5 = a_5 + 1
#     elif count == 6:
#         df_6 = pd.concat([df_6,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_6 = a_6 + 1
#     elif count == 7:
#         df_7 = pd.concat([df_7,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_7 = a_7 + 1
#     elif count == 8:
#         df_8 = pd.concat([df_8,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_8 = a_8 + 1
#     elif count == 9:
#         df_9 = pd.concat([df_9,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_9 = a_9 + 1
#     elif count == 10:
#         df_10 = pd.concat([df_10,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_10 = a_10 + 1
#     elif count > 10:
#         df_11 = pd.concat([df_11,df_only_less_40[df_only_less_40['SMILES'] ==smiles]], axis = 0).drop_duplicates()
#         a_11 = a_11 + 1

#### generate embeddings

In [23]:
import numpy as np
import pandas as pd
import esm
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t36_3B_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D-contact-regression.pt


In [21]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_0.shape[0]):
    seq_enzy = df_0['Protein sequence'].iloc[i]
    seq_smiles = df_0['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_0['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)

embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_0_occurance_df_0.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_0_occurance_df_0.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_0_occurance_df_0.pt')
print(embeddings_results_enzy_torch.shape)
print(embeddings_results_smiles_torch.shape)
print(embeddings_results_label_torch.shape)

torch.Size([331, 2560])
torch.Size([331, 768])
torch.Size([331, 1])


In [39]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_0.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_0_occurance_df_0.pt')
y_test = torch.load('ESP_test_df_label_0_occurance_df_0.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.18701924631992975 ACC=  0.7613293051359517 bacc=  0.605144149236857 precision=  0.6 specificity=  0.9344262295081968 sensitivity=  0.27586206896551724 recall=  0.27586206896551724 MCC=  0.28398353196087045 AUC=  0.6871584699453552 f1=  0.3779527559055118


<ipython-input-39-31c7237691dd>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_0.pt')
<ipython-input-39-31c7

(0.18701924631992975, 0.7613293051359517, 0.605144149236857, 228, 16, 63, 24)

In [44]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_1.shape[0]):
    seq_enzy = df_1['Protein sequence'].iloc[i]
    seq_smiles = df_1['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_1['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)

embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_0_occurance_df_1.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_0_occurance_df_1.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_0_occurance_df_1.pt')
print(embeddings_results_enzy_torch.shape)
print(embeddings_results_smiles_torch.shape)
print(embeddings_results_label_torch.shape)


torch.Size([442, 2560])
torch.Size([442, 768])
torch.Size([442, 1])


In [45]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_0_occurance_df_1.pt')
y_test = torch.load('ESP_test_df_label_0_occurance_df_1.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.0850506394635886 ACC=  0.9072398190045249 bacc=  0.8309504034171808 precision=  0.8607594936708861 specificity=  0.9680232558139535 sensitivity=  0.6938775510204082 recall=  0.6938775510204082 MCC=  0.7176592905632362 AUC=  0.9121381110583768 f1=  0.768361581920904


<ipython-input-45-00797ca7ac60>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
<ipython-input-45-0079

(0.0850506394635886, 0.9072398190045249, 0.8309504034171808, 333, 11, 30, 68)

In [ ]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_11.shape[0]):
    seq_enzy = df_11['Protein sequence'].iloc[i]
    seq_smiles = df_11['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_11['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)
        # print(seq_enzy, seq_smiles)
        # print(i)
embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_occurance_df_11.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_occurance_df_11.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_occurance_df_11.pt')


In [44]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_occurance_df_11.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_occurance_df_11.pt')
y_test = torch.load('ESP_test_df_label_occurance_df_11.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

<ipython-input-44-bdbee590a1a8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_occurance_df_11.pt')
<ipython-input-44-bdbee

loss_sum=  0.053861477853471035 ACC=  0.9425573537257692 bacc=  0.9060093738731783 precision=  0.954337899543379 specificity=  0.9853872381880175 sensitivity=  0.8266315095583389 recall=  0.8266315095583389 MCC=  0.8516916501778575 AUC=  0.9606470072415209 f1=  0.8859060402684563


(0.053861477853471035,
 0.9425573537257692,
 0.9060093738731783,
 4046,
 60,
 263,
 1254)

### df_only_40_60

#### split the dataset

In [31]:
# extract all the smiles
df_0 = pd.DataFrame()
df_1 = pd.DataFrame()
df_2 = pd.DataFrame()
df_3 = pd.DataFrame()
df_4 = pd.DataFrame()
df_5 = pd.DataFrame()
df_6 = pd.DataFrame()
df_7 = pd.DataFrame()
df_8 = pd.DataFrame()
df_9 = pd.DataFrame()
df_10 = pd.DataFrame()
df_11 = pd.DataFrame()
a = 0
a_0 = 0
a_1 = 0
a_2 = 0
a_3 = 0
a_4 = 0
a_5 = 0
a_6 = 0
a_7 = 0
a_8 = 0
a_9 = 0
a_10 = 0
a_11 = 0
for smiles in df_only_40_60['SMILES']:
    count = ESP_train_df[ESP_train_df['SMILES'] ==smiles].shape[0] # just check whether the smiles exist in the ESP_train dataset
    a = a + 1
    if count == 0: # means the smiles have not occur in the train dataset.
        df_0 = pd.concat([df_0,df_only_40_60[df_only_40_60['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_0 = a_0 + 1
    elif 0 < count <11 :
        df_1 = pd.concat([df_1,df_only_40_60[df_only_40_60['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_1 = a_1 + 1
    elif count > 10:
        df_11 = pd.concat([df_11,df_only_40_60[df_only_40_60['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_11 = a_11 + 1

In [32]:
print(df_0.shape, df_1.shape, df_11.shape)

(114, 6) (189, 6) (2405, 6)


#### generate embeddings

In [47]:
import numpy as np
import pandas as pd
import esm
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()

In [48]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_0.shape[0]):
    seq_enzy = df_0['Protein sequence'].iloc[i]
    seq_smiles = df_0['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_0['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)

embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_0_occurance_df_0.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_0_occurance_df_0.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_0_occurance_df_0.pt')
print(embeddings_results_enzy_torch.shape)
print(embeddings_results_smiles_torch.shape)
print(embeddings_results_label_torch.shape)

torch.Size([114, 2560])
torch.Size([114, 768])
torch.Size([114, 1])


In [49]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_0.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_0_occurance_df_0.pt')
y_test = torch.load('ESP_test_df_label_0_occurance_df_0.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.166885232552886 ACC=  0.8508771929824561 bacc=  0.640625 precision=  0.5454545454545454 specificity=  0.9479166666666666 sensitivity=  0.3333333333333333 recall=  0.3333333333333333 MCC=  0.34733571745190384 AUC=  0.6660879629629629 f1=  0.41379310344827586


<ipython-input-49-31c7237691dd>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_0.pt')
<ipython-input-49-31c7

(0.166885232552886, 0.8508771929824561, 0.640625, 91, 5, 12, 6)

In [33]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []

for i in range(df_1.shape[0]):
    seq_enzy = df_1['Protein sequence'].iloc[i]
    seq_smiles = df_1['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_1['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)

embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_0_occurance_df_1.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_0_occurance_df_1.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_0_occurance_df_1.pt')
print(embeddings_results_enzy_torch.shape)
print(embeddings_results_smiles_torch.shape)
print(embeddings_results_label_torch.shape)


torch.Size([189, 2560])
torch.Size([189, 768])
torch.Size([189, 1])


In [39]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_0_occurance_df_1.pt')
y_test = torch.load('ESP_test_df_label_0_occurance_df_1.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.07222734267512958 ACC=  0.9259259259259259 bacc=  0.8649448754593712 precision=  0.7741935483870968 specificity=  0.9556962025316456 sensitivity=  0.7741935483870968 recall=  0.7741935483870968 MCC=  0.7298897509187423 AUC=  0.9007758268681094 f1=  0.7741935483870968


<ipython-input-39-00797ca7ac60>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
<ipython-input-39-0079

(0.07222734267512958, 0.9259259259259259, 0.8649448754593712, 151, 7, 7, 24)

In [51]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_11.shape[0]):
    seq_enzy = df_11['Protein sequence'].iloc[i]
    seq_smiles = df_11['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_11['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)
        # print(seq_enzy, seq_smiles)
        # print(i)
embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_occurance_df_11.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_occurance_df_11.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_occurance_df_11.pt')


[23:28:00] WARNING: not removing hydrogen atom without neighbors
[23:28:00] WARNING: not removing hydrogen atom without neighbors
[23:28:00] WARNING: not removing hydrogen atom without neighbors


In [52]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_occurance_df_11.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_occurance_df_11.pt')
y_test = torch.load('ESP_test_df_label_occurance_df_11.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

<ipython-input-52-bdbee590a1a8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_occurance_df_11.pt')
<ipython-input-52-bdbee

loss_sum=  0.035024682818511066 ACC=  0.9692307692307692 bacc=  0.9534327301821763 precision=  0.971830985915493 specificity=  0.9895833333333334 sensitivity=  0.9172821270310192 recall=  0.9172821270310192 MCC=  0.9233434782346726 AUC=  0.9837774905629411 f1=  0.9437689969604863


(0.035024682818511066,
 0.9692307692307692,
 0.9534327301821763,
 1710,
 18,
 56,
 621)

### df_only_60_80

#### split the dataset

In [46]:
# extract all the smiles
df_0 = pd.DataFrame()
df_1 = pd.DataFrame()
df_2 = pd.DataFrame()
df_3 = pd.DataFrame()
df_4 = pd.DataFrame()
df_5 = pd.DataFrame()
df_6 = pd.DataFrame()
df_7 = pd.DataFrame()
df_8 = pd.DataFrame()
df_9 = pd.DataFrame()
df_10 = pd.DataFrame()
df_11 = pd.DataFrame()
a = 0
a_0 = 0
a_1 = 0
a_2 = 0
a_3 = 0
a_4 = 0
a_5 = 0
a_6 = 0
a_7 = 0
a_8 = 0
a_9 = 0
a_10 = 0
a_11 = 0
for smiles in df_only_60_80['SMILES']:
    count = ESP_train_df[ESP_train_df['SMILES'] ==smiles].shape[0] # just check whether the smiles exist in the ESP_train dataset
    a = a + 1
    if count == 0: # means the smiles have not occur in the train dataset.
        df_0 = pd.concat([df_0,df_only_60_80[df_only_60_80['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_0 = a_0 + 1
    elif 0 < count <11 :
        df_1 = pd.concat([df_1,df_only_60_80[df_only_60_80['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_1 = a_1 + 1
    elif count > 10:
        df_11 = pd.concat([df_11,df_only_60_80[df_only_60_80['SMILES'] ==smiles]], axis = 0).drop_duplicates()
        a_11 = a_11 + 1

In [47]:
print(df_0.shape, df_1.shape, df_11.shape)

(67, 6) (116, 6) (1470, 6)


#### generate embeddings

In [55]:
import numpy as np
import pandas as pd
import esm
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()

In [56]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_0.shape[0]):
    seq_enzy = df_0['Protein sequence'].iloc[i]
    seq_smiles = df_0['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_0['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)

embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_0_occurance_df_0.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_0_occurance_df_0.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_0_occurance_df_0.pt')
print(embeddings_results_enzy_torch.shape)
print(embeddings_results_smiles_torch.shape)
print(embeddings_results_label_torch.shape)

torch.Size([67, 2560])
torch.Size([67, 768])
torch.Size([67, 1])


In [57]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_0.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_0_occurance_df_0.pt')
y_test = torch.load('ESP_test_df_label_0_occurance_df_0.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.08364484123885632 ACC=  0.8955223880597015 bacc=  0.6165254237288136 precision=  0.6666666666666666 specificity=  0.9830508474576272 sensitivity=  0.25 recall=  0.25 MCC=  0.36540212737375455 AUC=  0.7161016949152542 f1=  0.36363636363636365


<ipython-input-57-31c7237691dd>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_0.pt')
<ipython-input-57-31c7

(0.08364484123885632, 0.8955223880597015, 0.6165254237288136, 58, 1, 6, 2)

In [58]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
if df_1.empty:
    print(' this group is empty')
else:
    for i in range(df_1.shape[0]):
        seq_enzy = df_1['Protein sequence'].iloc[i]
        seq_smiles = df_1['SMILES'].iloc[i]
        if len(seq_enzy) < 5500:
            # print(len(seq_enzy))
            tuple_sequence = tuple(['protein',seq_enzy])
            peptide_sequence_list = []
            peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
            # employ ESM model for converting and save the converted data in csv format
            one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
            embeddings_results_enzy.append(one_seq_embeddings)
            # the smiles embeddings
            smiles_list = []
            smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
            # employ ESM model for converting and save the converted data in csv format
            one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
            embeddings_results_smiles.append(one_seq_embeddings)
            # record the lable info
            label = torch.tensor(df_1['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
            embeddings_results_label.append(label)

    embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
    torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_0_occurance_df_1.pt')

    embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
    torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_0_occurance_df_1.pt')

    embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
    torch.save(embeddings_results_label_torch, 'ESP_test_df_label_0_occurance_df_1.pt')
    print(embeddings_results_enzy_torch.shape)
    print(embeddings_results_smiles_torch.shape)
    print(embeddings_results_label_torch.shape)
    import torch
    from torch.utils.data import TensorDataset, DataLoader
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_0_occurance_df_1.pt')
    y_test = torch.load('ESP_test_df_label_0_occurance_df_1.pt')
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
    # Create TensorDataset and DataLoaders
    batch_size = 16
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

    run_validation(model_test,test_loader,loss_fn, device)

torch.Size([116, 2560])
torch.Size([116, 768])
torch.Size([116, 1])
loss_sum=  0.043214541277848184 ACC=  0.9655172413793104 bacc=  0.949010989010989 precision=  0.92 specificity=  0.978021978021978 sensitivity=  0.92 recall=  0.92 MCC=  0.898021978021978 AUC=  0.956043956043956 f1=  0.92


<ipython-input-58-a894bb7ef943>:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
<ipython-input-58-a89

In [50]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []

for i in range(df_1.shape[0]):
    seq_enzy = df_1['Protein sequence'].iloc[i]
    seq_smiles = df_1['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_1['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)

embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_0_occurance_df_1.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_0_occurance_df_1.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_0_occurance_df_1.pt')
print(embeddings_results_enzy_torch.shape)
print(embeddings_results_smiles_torch.shape)
print(embeddings_results_label_torch.shape)



torch.Size([116, 2560])
torch.Size([116, 768])
torch.Size([116, 1])


In [51]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_0_occurance_df_1.pt')
y_test = torch.load('ESP_test_df_label_0_occurance_df_1.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.043214541277848184 ACC=  0.9655172413793104 bacc=  0.949010989010989 precision=  0.92 specificity=  0.978021978021978 sensitivity=  0.92 recall=  0.92 MCC=  0.898021978021978 AUC=  0.956043956043956 f1=  0.92


<ipython-input-51-00797ca7ac60>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_0_occurance_df_1.pt')
<ipython-input-51-0079

(0.043214541277848184, 0.9655172413793104, 0.949010989010989, 89, 2, 2, 23)

In [52]:
# generate the peptide embeddings
embeddings_results_enzy = []
embeddings_results_smiles = []
embeddings_results_label = []
for i in range(df_11.shape[0]):
    seq_enzy = df_11['Protein sequence'].iloc[i]
    seq_smiles = df_11['SMILES'].iloc[i]
    if len(seq_enzy) < 5500:
        # print(len(seq_enzy))
        tuple_sequence = tuple(['protein',seq_enzy])
        peptide_sequence_list = []
        peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
        embeddings_results_enzy.append(one_seq_embeddings)
        # the smiles embeddings
        smiles_list = []
        smiles_list.append(Chem.CanonSmiles(seq_smiles)) # build a summarize list variable including all the sequence information
        # employ ESM model for converting and save the converted data in csv format
        one_seq_embeddings = MolFormer_embedding(model_smiles, tokenizer, smiles_list)
        embeddings_results_smiles.append(one_seq_embeddings)
        # record the lable info
        label = torch.tensor(df_11['output'].iloc[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        embeddings_results_label.append(label)
        # print(seq_enzy, seq_smiles)
        # print(i)
embeddings_results_enzy_torch = torch.cat(embeddings_results_enzy, dim=0)
torch.save(embeddings_results_enzy_torch, 'ESP_test_df_enzy_occurance_df_11.pt')

embeddings_results_smiles_torch = torch.cat(embeddings_results_smiles, dim=0)
torch.save(embeddings_results_smiles_torch, 'ESP_test_df_smiles_occurance_df_11.pt')

embeddings_results_label_torch = torch.cat(embeddings_results_label, dim=0)
torch.save(embeddings_results_label_torch, 'ESP_test_df_label_occurance_df_11.pt')


[00:50:15] WARNING: not removing hydrogen atom without neighbors


In [53]:
import torch
from torch.utils.data import TensorDataset, DataLoader
ESP_test_df_enzy = torch.load('ESP_test_df_enzy_occurance_df_11.pt')
ESP_test_df_smiles = torch.load('ESP_test_df_smiles_occurance_df_11.pt')
y_test = torch.load('ESP_test_df_label_occurance_df_11.pt')
test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)
# Create TensorDataset and DataLoaders
batch_size = 16
test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)

run_validation(model_test,test_loader,loss_fn, device)

loss_sum=  0.030616079897457814 ACC=  0.972108843537415 bacc=  0.9616342504743833 precision=  0.9629629629629629 specificity=  0.9857685009487666 sensitivity=  0.9375 recall=  0.9375 MCC=  0.9308778023983193 AUC=  0.992353990293388 f1=  0.9500609013398295


<ipython-input-53-bdbee590a1a8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ESP_test_df_enzy = torch.load('ESP_test_df_enzy_occurance_df_11.pt')
<ipython-input-53-bdbee

(0.030616079897457814,
 0.972108843537415,
 0.9616342504743833,
 1039,
 15,
 26,
 390)